References:

https://docs.azuredatabricks.net/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html<br>
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame<br>
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.coalesce<br>
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.repartition<br>

Coalesce: Returns a new DataFrame that has exactly numPartitions partitions.<br>

Repartition: Returns a new DataFrame partitioned by the given partitioning expressions. The resulting DataFrame is hash partitioned.<br>

In [2]:
%run ./adb_3_ingest_to_df

### Partitions and repartitioning

In [4]:
num_partitions = 4

In [5]:
print(df_explicit.rdd.getNumPartitions())

In [6]:
df2 = df_explicit.repartition(num_partitions, "state")

In [7]:
# How many partitions were created due to our partitioning expression

print(df2.rdd.getNumPartitions())

### Parquet

In [9]:
parquet_path = "/mnt/hack/parquet/sample/dat202/"

In [10]:
path_coalesce = parquet_path + "coalesce/"
dbutils.fs.rm(path_coalesce, True)

In [11]:
# coalesce(numPartitions: Int): DataFrame - Returns a new DataFrame that has exactly numPartitions partitions

df_explicit\
  .coalesce(num_partitions)\
  .write\
  .parquet(path_coalesce)

In [12]:
path_repartition = parquet_path + "repartition/"
dbutils.fs.rm(path_repartition, True)

In [13]:
# Repartition and write

df_explicit\
  .repartition(num_partitions, "state")\
  .write\
  .parquet(path_repartition)

In [14]:
path_repartition_partitionby = parquet_path + "repartition-partitionby/"
dbutils.fs.rm(path_repartition_partitionby, True)

In [15]:
# Repartition and write. Here, we are partitioning the output (write.partitionBy) which will create a folder per value in the partition field
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.repartition

df_explicit\
  .repartition(num_partitions, "state")\
  .write\
  .partitionBy("state")\
  .parquet(path_repartition_partitionby)